In [1]:
import os
import random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(5)

In [2]:
import pandas as pd
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [5]:
import numpy as np
import keras
from scipy.io import loadmat
train = loadmat('train.mat')
test = loadmat('test.mat')
X_train = train['X']
y_train = train['y']
X_test = test['X']
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train[np.newaxis,...]
X_train = np.swapaxes(X_train,0,4).squeeze()
X_test = X_test[np.newaxis,...]
X_test = np.swapaxes(X_test,0,4).squeeze()


np.place(y_train,y_train == 10,0)

y_train = keras.utils.to_categorical(y_train, 10)
# ================================================

# Create train and test for training model
Xtrain = X_train[1:65931]
Ytrain = y_train[1:65931]
Xtest = X_train[65931:]
Ytest = y_train[65931:]

In [6]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)
datagen.fit(Xtrain, augment=True)

In [8]:
model1 = load_model('weights.best.wrn_52-1.hdf5')
prob_y1 = model1.predict_generator(datagen.flow(X_test,batch_size=128, shuffle=False), workers=16, use_multiprocessing=True)

/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1269: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [9]:
model2 = load_model('weights.best.wrn_34-2.hdf5')
prob_y2 = model2.predict_generator(datagen.flow(X_test,batch_size=128, shuffle=False), workers=16, use_multiprocessing=True)

/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1269: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [ ]:
prob_all = prob_y1 + prob_y2
pred_y = prob_all.argmax(axis=-1)
np.place(pred_y,pred_y == 0,10)

In [20]:
import pandas as pd
table = pd.DataFrame()
pred_id = list(range(1,X_test.shape[0]+1))
table.insert(0, 'id', pred_id)
table.insert(1, 'class', pred_y)
table.to_csv("WideResnet" + '_predictions.csv', sep=',', index=False)